In [1]:
__authors__ = ["Simone Bavera <Simone.Bavera@unige.ch>"]

In [2]:
import os
import csv
import numpy as np
import pandas as pd

In [3]:
def log_range(x_min,x_max,x_n):
    return 10**np.linspace(np.log10(x_min),np.log10(x_max), x_n)
def lin_range(x_min,x_max,x_n):
    return np.linspace(x_min,x_max, x_n)

In [4]:
# digit rounding pick 10 to be sure we resolve 10^-4Zsun
NDIG = 10

# HMS-HMS

In [22]:
Zsun = 0.0142
m1_min = 5.550812451317292
m1_max = 303.4919603386519
m1_n = 70
m1 = log_range(m1_min,m1_max,m1_n)
q_min = 0.05
q_max = 1.
q_n = 20
q = lin_range(q_min,q_max,q_n)
q[-1] = 0.99 # replace q=1 with 0.99
p_min = 10**(-1.)
p_max = 6105.402296585326
p_n = 68
p = log_range(p_min,p_max,p_n)
Z_min = 10**(-4)
Z_max = 10**0
Z_n = 5
met = log_range(Z_min,Z_max,Z_n)*Zsun
# metallicity extentions
met = np.array([2.,0.2])*Zsun
met = np.array([0.45])*Zsun # changed %0.e_Zsun in favour of %0.1e_Zsun
print('total high resolution  Z_n * m1_n * q_n * p_n=', Z_n * m1_n * q_n * p_n)
print('reduced resolution  Z_n * m1_n/2 * q_n * p_n/2 = ',Z_n * m1_n/2 * q_n * p_n/2)
print('reduced resolution per metallicity m1_n/2 * q_n * p_n/2 =',m1_n/2 * q_n * p_n/2)

total high resolution  Z_n * m1_n * q_n * p_n= 476000
reduced resolution  Z_n * m1_n/2 * q_n * p_n/2 =  119000.0
reduced resolution per metallicity m1_n/2 * q_n * p_n/2 = 23800.0


In [23]:
for Z in met:
    # save entire grid in a single file
    with open('./HMS-HMS/%0.1e_Zsun/grid_high_res_all.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','m1','m2','initial_period_in_days'])
        for i in range(m1_n):
            for j in range(q_n):
                for k in range(p_n):
                    if m1[i]*q[j] >= 0.5:
                        writer.writerow([round(Z,NDIG),round(Z,NDIG),round(m1[i],NDIG),round(m1[i]*q[j],NDIG),round(p[k],NDIG)])
    # create reduced resolution grid and split into files of max 4000 points
    df = pd.read_csv('./HMS-HMS/%0.1e_Zsun/grid_high_res_all.csv'%(Z/Zsun)) 
    sel = df['m1'].isin(np.around(m1,NDIG)[::2]) & df['initial_period_in_days'].isin(np.around(p,NDIG)[::2])
    df = df[sel]
    size = 4000
    files = int(len(df)/size)+1
    chunks = [df[i*size:(i+1)*size] for i in range(files)]
    for i, chunk in enumerate(chunks):
        chunk.to_csv('./HMS-HMS/%0.1e_Zsun/grid_low_res_%i.csv'%(Z/Zsun,i),index=False)
    # create high resolution grid (without reduced res points) and split into files of max 4000 points
    # df = pd.read_csv('./HMS-HMS/%0.e_Zsun/grid_high_res_all.csv'%(Z/Zsun)) 
    # sel = df['m1'].isin(np.around(m1,NDIG)[::2]) & df['initial_period_in_days'].isin(np.around(p,NDIG)[::2])
    # df = df[~sel]
    # size = 4000
    # files = int(len(df)/size)+1
    # chunks = [df[i*size:(i+1)*size] for i in range(files)]
    # for i, chunk in enumerate(chunks):
    #     chunk.to_csv('./HMS-HMS/%0.e_Zsun/grid_high_res_%i.csv'%(Z/Zsun,i),index=False)

In [24]:
# random grid
n = 3000
for Z in met:
    m1 = 10**np.random.uniform(np.log10(m1_min), np.log10(m1_max), n)
    q = np.random.uniform(q_min, q_max, n)
    p = 10**np.random.uniform(np.log10(p_min),np.log10(p_max), n)
    with open('./HMS-HMS/%0.1e_Zsun/grid_random_1.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','m1','m2','initial_period_in_days'])
        for j in range(n):
            if m1[j]*q[j] >= 0.5:
                writer.writerow([round(Z,NDIG),round(Z,NDIG),round(m1[j],NDIG),round(m1[j]*q[j],NDIG),round(p[j],NDIG)])

In [25]:
# test grid
n = 10
for Z in met:
    m1 = 10**np.random.uniform(np.log10(m1_min), np.log10(m1_max), n)
    q = np.random.uniform(q_min, q_max, n)
    p = 10**np.random.uniform(np.log10(p_min),np.log10(p_max), n)
    with open('./HMS-HMS/%0.1e_Zsun/grid_test.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','m1','m2','initial_period_in_days'])
        for j in range(n):
            if m1[j]*q[j] >= 0.5:
                writer.writerow([round(Z,NDIG),round(Z,NDIG),round(m1[j],NDIG),round(m1[j]*q[j],NDIG),round(p[j],NDIG)])

# CO-HMS_RLO

In [30]:
Zsun = 0.0142
m_CO_min = 1.
m_CO_max = 51.32759630397827
m_CO_n = 23
m_CO = log_range(m_CO_min, m_CO_max, m_CO_n)
m_star_min = 0.5
m_star_max = 320.92137804463766
m_star_n = 47
m_star = log_range(m_star_min, m_star_max, m_star_n)
p_min = 10**(-1)
p_max = 10**(3.5)
p_n = 30
p = log_range(p_min,p_max, p_n)
Z_min = 10**(-4)
Z_max = 10**0
Z_n = 5
met = log_range(Z_min,Z_max,Z_n)*Zsun
# metallicity extentions
met = np.array([2.,0.2])*Zsun
met = np.array([0.45])*Zsun # changed %0.e_Zsun in favour of %0.1e_Zsun
print('total high resolution  Z_n * m_star_n * m_CO_n * p_n=', Z_n * m_star_n * m_CO_n * p_n)
print('reduced resolution  Z_n * m_star_n/2 * m_CO_n * p_n/2 = ',Z_n * (m_star_n+1)/2 * m_CO_n * p_n/2)
print('reduced resolution per metallicity m_star_n/2 * m_CO_n * p_n/2 =',(m_star_n+1)/2 * m_CO_n * p_n/2)

total high resolution  Z_n * m_star_n * m_CO_n * p_n= 162150
reduced resolution  Z_n * m_star_n/2 * m_CO_n * p_n/2 =  41400.0
reduced resolution per metallicity m_star_n/2 * m_CO_n * p_n/2 = 8280.0


In [33]:
for Z in met:
    # save entire grid in a single file
    with open('./CO-HMS_RLO/%0.1e_Zsun/grid_high_res_all.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','m1','m2','initial_period_in_days'])
        for i in range(m_CO_n):
            for j in range(m_star_n):
                for k in range(p_n):
                    writer.writerow([round(Z,NDIG),round(Z,NDIG),round(m_star[j],NDIG),round(m_CO[i],NDIG),round(p[k],NDIG)])
    # create reduced resolution grid and split into files of max 4000 points
    df = pd.read_csv('./CO-HMS_RLO/%0.1e_Zsun/grid_high_res_all.csv'%(Z/Zsun)) 
    sel = df['m1'].isin(np.around(m_star,NDIG)[::2]) & df['initial_period_in_days'].isin(np.around(p,NDIG)[::2])
    df = df[sel]
    size = 4000
    files = int(len(df)/size)+1
    chunks = [df[i*size:(i+1)*size] for i in range(files)]
    for i, chunk in enumerate(chunks):
        chunk.to_csv('./CO-HMS_RLO/%0.1e_Zsun/grid_low_res_%i.csv'%(Z/Zsun,i),index=False)
    # create high resolution grid (without reduced res points) and split into files of max 4000 points
    # df = pd.read_csv('./CO-HMS_RLO/%0.e_Zsun/grid_high_res_all.csv'%(Z/Zsun)) 
    # sel = df['m1'].isin(np.around(m_star,NDIG)[::2]) & df['initial_period_in_days'].isin(np.around(p,NDIG)[::2])
    # df = df[~sel]
    # size = 4000
    # files = int(len(df)/size)+1
    # chunks = [df[i*size:(i+1)*size] for i in range(files)]
    # for i, chunk in enumerate(chunks):
    #     chunk.to_csv('./CO-HMS_RLO/%0.e_Zsun/grid_high_res_%i.csv'%(Z/Zsun,i),index=False)

In [34]:
#random 
n = 3000
for Z in met:
    m_CO = 10**np.random.uniform(np.log10(m_CO_min),np.log10(m_CO_max), n)
    m_star = 10**np.random.uniform(np.log10(m_star_min),np.log10(m_star_max), n)
    p = 10**np.random.uniform(np.log10(p_min),np.log10(p_max), n)
    with open('./CO-HMS_RLO/%0.1e_Zsun/grid_random_1.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','m1','m2','initial_period_in_days'])
        for j in range(n):
            writer.writerow([round(Z,NDIG),round(Z,NDIG),round(m_star[j],NDIG),round(m_CO[j],NDIG),round(p[j],NDIG)])

In [35]:
#random 
n = 10
for Z in met:
    m_CO = 10**np.random.uniform(np.log10(m_CO_min),np.log10(m_CO_max), n)
    m_star = 10**np.random.uniform(np.log10(m_star_min),np.log10(m_star_max), n)
    p = 10**np.random.uniform(np.log10(p_min),np.log10(p_max), n)
    with open('./CO-HMS_RLO/%0.1e_Zsun/grid_test.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','m1','m2','initial_period_in_days'])
        for j in range(n):
            writer.writerow([round(Z,NDIG),round(Z,NDIG),round(m_star[j],NDIG),round(m_CO[j],NDIG),round(p[j],NDIG)])

# CO-HeMS

In [40]:
Zsun = 0.0142
m_CO_min = 1.
m_CO_max = 51.32759630397827
m_CO_n = 23
m_CO = log_range(m_CO_min,m_CO_max,m_CO_n)
m_star_min = 0.5
m_star_max = 191.91577479071924
m_star_n = 35
m_star = log_range(m_star_min,m_star_max,m_star_n)
p_min = 0.02
p_max = 1147.2305020897365
p_n = 47
p = log_range(p_min,p_max,p_n)
Z_min = 10**(-4)
Z_max = 10**0
Z_n = 5
met = log_range(Z_min,Z_max,Z_n)*Zsun
# metallicity extentions
met = np.array([2.,0.2])*Zsun
met = np.array([0.45])*Zsun # changed %0.e_Zsun in favour of %0.1e_Zsun
print('total high resolution  Z_n * m_star_n * m_CO_n * p_n=', Z_n * m_star_n * m_CO_n * p_n)
print('reduced resolution  Z_n * m_star_n/2 * m_CO_n * p_n/2 = ',Z_n * (m_star_n+1)/2 * m_CO_n * p_n/2)
print('reduced resolution per metallicity m_star_n/2 * m_CO_n * p_n/2 =',(m_star_n+1)/2 * m_CO_n * p_n/2)

total high resolution  Z_n * m_star_n * m_CO_n * p_n= 189175
reduced resolution  Z_n * m_star_n/2 * m_CO_n * p_n/2 =  48645.0
reduced resolution per metallicity m_star_n/2 * m_CO_n * p_n/2 = 9729.0


In [41]:
for Z in met:
    # save entire grid in a single file
    with open('./CO-HeMS/%0.1e_Zsun/grid_high_res_all.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','new_Z','m1','m2','initial_period_in_days'])
        for i in range(m_CO_n):
            for j in range(m_star_n):
                for k in range(p_n):
                    writer.writerow([round(Z,NDIG),round(Z,NDIG),round(Z,NDIG),round(m_star[j],NDIG),round(m_CO[i],NDIG),round(p[k],NDIG)])
    # create reduced resolution grid and split into files of max 4000 points
    df = pd.read_csv('./CO-HeMS/%0.1e_Zsun/grid_high_res_all.csv'%(Z/Zsun)) 
    sel = df['m1'].isin(np.around(m_star,NDIG)[::2]) & df['initial_period_in_days'].isin(np.around(p,NDIG)[::2])
    df = df[sel]
    size = 4000
    files = int(len(df)/size)+1
    chunks = [df[i*size:(i+1)*size] for i in range(files)]
    for i, chunk in enumerate(chunks):
        chunk.to_csv('./CO-HeMS/%0.1e_Zsun/grid_low_res_%i.csv'%(Z/Zsun,i),index=False)
    # create high resolution grid (without reduced res points) and split into files of max 4000 points
    # df = pd.read_csv('./CO-HeMS/%0.e_Zsun/grid_high_res_all.csv'%(Z/Zsun)) 
    # sel = df['m1'].isin(np.around(m_star,NDIG)[::2]) & df['initial_period_in_days'].isin(np.around(p,NDIG)[::2])
    # df = df[~sel]
    # size = 4000
    # files = int(len(df)/size)+1
    # chunks = [df[i*size:(i+1)*size] for i in range(files)]
    # for i, chunk in enumerate(chunks):
    #     chunk.to_csv('./CO-HeMS/%0.e_Zsun/grid_high_res_%i.csv'%(Z/Zsun,i),index=False)

In [42]:
# random grid
n = 3000
for Z in met:
    m_CO = 10**np.random.uniform(np.log10(m_CO_min),np.log10(m_CO_max), n)
    m_star = 10**np.random.uniform(np.log10(m_star_min),np.log10(m_star_max), n)
    p = 10**np.random.uniform(np.log10(p_min),np.log10(p_max), n)
    with open('./CO-HeMS/%0.1e_Zsun/grid_random_1.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','new_Z','m1','m2','initial_period_in_days'])
        for j in range(n):
            writer.writerow([round(Z,NDIG),round(Z,NDIG),round(Z,NDIG),round(m_star[j],NDIG),round(m_CO[j],NDIG),round(p[j],NDIG)])

In [43]:
# test grid
n = 10
for Z in met:
    m_CO = 10**np.random.uniform(np.log10(m_CO_min),np.log10(m_CO_max), n)
    m_star = 10**np.random.uniform(np.log10(m_star_min),np.log10(m_star_max), n)
    p = 10**np.random.uniform(np.log10(p_min),np.log10(p_max), n)
    with open('./CO-HeMS/%0.1e_Zsun/grid_test.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','new_Z','m1','m2','initial_period_in_days'])
        for j in range(n):
            writer.writerow([round(Z,10),round(Z,10),round(Z,10),round(m_star[j],NDIG),round(m_CO[j],NDIG),round(p[j],NDIG)])

# single HMS

In [38]:
Zsun = 0.0142
m1_min = 0.1
m1_max = 300.
m1_n = 375
m1 = log_range(m1_min,m1_max,m1_n)

Z_min = 10**(-4)
Z_max = 10**0
Z_n = 5
met = log_range(Z_min,Z_max,Z_n)*Zsun
# metallicity extentions
met = np.array([2.,0.2])*Zsun
met = np.array([0.45])*Zsun # changed %0.e_Zsun in favour of %0.1e_Zsun

In [39]:
for Z in met:
    # save entire grid in a single file
    with open('./single_HMS/%0.1e_Zsun/grid.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','initial_mass'])
        for i in range(m1_n):
            writer.writerow([round(Z,NDIG),round(Z,NDIG),round(m1[i],NDIG)])

# single HeMS

In [52]:
Zsun = 0.0142
m1_min = 0.5
m1_max = 300.
m1_n = 300
m1 = log_range(m1_min,m1_max,m1_n)[:268]
m1_n = len(m1)

Z_min = 10**(-4)
Z_max = 10**0
Z_n = 5
met = log_range(Z_min,Z_max,Z_n)*Zsun
# metallicity extentions
met = np.array([2.,0.2])*Zsun
met = np.array([0.45])*Zsun # changed %0.e_Zsun in favour of %0.1e_Zsun

In [53]:
for Z in met:
    # save entire grid in a single file
    with open('./single_HeMS/%0.1e_Zsun/grid.csv'%(Z/Zsun), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','new_Z','initial_mass'])
        for i in range(m1_n):
            writer.writerow([round(Z,NDIG),round(Z,NDIG),round(Z,NDIG),round(m1[i],NDIG)])